# Bluesky Data Collection Framework

Callum Forrester

## Bluesky

- Family of software libraries developed by NSLS-II in Brookhaven, New York, USA.
- Confusingly, also the name of one of those libraries!
- Analysis-first data collection
- Aimed at making it easy to programatically make a beamline produce diverse data/metadata to suit diverse analysis needs

<img src="img/logo-bluesky.png" width="500"/>

## NSLS-II
- Operational since 2015
- Storage ring nearly 800m in circumference

<img src="img/nsls2.png" width="500"/>

## Diamond Almost Fits Inside...

<img src="img/nsls2-dls.png" width="500"/>

## Bread & Butter of Data Collection

- Control environment with actuators, record data with sensors/detectors
- Classic example, move sample through beam with motors, take pictures of interfered beam with detector:

<img src="img/scanning.svg" width="1000"/>
<img src="img/graph.svg" width="500"/>

## Beamline Software Stack

<img src="img/stack.svg" width="500"/>

## A Good DCS Should be...

- Versatile
    - Produce a wide variety of data
    - Perform modular tasks that can be composed in different ways, no two experiments are the same
- Scriptable in a scientist-friendly way
    - Python
    - Minimal boilerplate
    - Balance of simplicity and expressiveness: beamlines are complicated
- Balance of stability and innovation
    - Provide common, plug-and-play functionality where possible
    - Easy to write bespoke functionality where necessary
    - Innovation happens in-situ, but also need a stable platform to fall back to to satify user demand
    - Spectrum of users, some want a library, some want a big button that says go:

<img src="img/spectrum.svg" width="500"/>
    

## Bluesky

- Collection of components
- Loosely coupled
- Python libraries
- Friendly to people who know the Python scientific ecosystem, numpy, scipy etc.

<img src="img/bluesky.png" width="500"/>

- Building blocks:
    - Devices: Python abstractions of motors, detectors, etc.
    - Plans: Instructions for sequencing complex actions
    - Run Engine: Control object for running plans
    - Events: Descriptions of data recorded, emitted by run engine

## Devices

- Defined interfaces
    - `Readable`
    - `Movable`
    - `Flyable`
    - `Stageable`
    - `Checkable`
- Can have any device library that follows these interfaces
- NSLS-II have one called Ophyd (EPICS focused), there is nothing to stop you from writing your own

In [ ]:
from ophyd.sim import SynAxis, Syn2DGauss

x = SynAxis(name="x", delay=0.1)
y = SynAxis(name="y", delay=0.1)
det = Syn2DGauss(
    "det",
    x,
    "x",
    y,
    "y",
    center=(0, 0),
    Imax=1,
    labels={"detectors"},
)


In [ ]:
print(f"x and y are at ({x.position}, {y.position})")

x.set(x.position + 10).wait()
y.set(y.position + 5).wait()

print(f"x and y are now at ({x.position}, {y.position})")

## Plans

- Iterable sequence of messages, instructions on what the beamline should do and what data should be recorded
- The following are all valid plans:

In [ ]:
from typing import Iterable
from bluesky.utils import Msg


# Empty plan:
[]

# Plan with two messages:
[Msg("open_run"), Msg("close_run")]

# Dyanmic plan
def my_plan() -> Iterable[Msg]:
    yield Msg("open_run")
    if 3 < 4:
        yield Msg("checkpoint")
    yield Msg("close_run")

list(my_plan())

## Events

- Emitted when data recorded
- Structured into "runs" 

In [ ]:
from typing import Dict

import bluesky.plan_stubs as bps
import bluesky.plans as bp

from bluesky import RunEngine
from pprint import pprint

def print_event(name: str, doc: Dict) -> None:
    pprint({name: doc})

RE = RunEngine()
RE(bp.count([det]), print_event)

## Scanning

- Change some variables, read other variables
- Sample many points

In [ ]:
%matplotlib notebook 

from bluesky.callbacks.best_effort import BestEffortCallback

RE(bp.scan([det], x, -10, 10, 10), BestEffortCallback())

## Custom Plans

- Easy to write your own
- Meant for doing bespoke science

In [ ]:
def zoom_in(start_x: float, start_y: float, stop_x: float, stop_y: float, res: int):
    # Scan given area
    yield from bp.grid_scan([det], x, start_x, stop_x, res, y, start_y, stop_y, res)
    
    # Zoomed-in version
    start_x += (stop_x - start_x) / 4
    start_y += (stop_y - start_y) / 4
    stop_x -= (stop_x - start_x) / 4
    stop_y -= (stop_y - start_y) / 4
    yield from bp.grid_scan([det], x, start_x, stop_x, res, y, start_y, stop_y, res)

RE(zoom_in(-2, -2, 2, 2, 10), BestEffortCallback())

## Adaptive Plans and Automation

- Plans can be made to react to measurement results or changes in environment
- Potential use for creating complicated, bespoke workflows
- Interesting potential for beamline automation
- MX particularly interested in this

https://github.com/DiamondLightSource/python-artemis


In [ ]:
import time
from math import sin
from ophyd.sim import SynPeriodicSignal

# Create simulated temperature readout
def sine_temp() -> float:
    return sin(time.time()) * 10 + 15

temperature = SynPeriodicSignal(sine_temp, name="temperature")
temperature.start_simulation()

In [ ]:
from bluesky.callbacks import LivePlot

RE(bp.count([temperature], num=50, delay=0.1), LivePlot("temperature"))

In [ ]:
from typing import List
from dataclasses import dataclass

from bluesky.preprocessors import monitor_during_decorator, suspend_decorator
from bluesky.suspenders import SuspendWhenOutsideBand

@dataclass
class Instruction:
    start: float
    stop: float
    steps: int

def my_experiment(scans_to_do: List[Instruction], min_temp: float, max_temp: float):
    suspender = SuspendWhenOutsideBand(temperature, min_temp, max_temp)

    @suspend_decorator([suspender])
    @monitor_during_decorator([temperature])
    def do_scans():
        for instruction in scans_to_do:
            yield from bp.scan([det], x, instruction.start, instruction.stop, instruction.steps)

    return (yield from do_scans())


In [ ]:
scans = [
    Instruction(-20., 20., 50),
    Instruction(-5., 5., 50),
    Instruction(7., 10., 20),
    Instruction(-9., -8., 20),
]

RE(my_experiment(scans, 10., 22.), LivePlot("det", "x"))

## Comparison with Diamond's Data Collection System

- GDA
- Java application with a Jython interpreter for scripting
- GUIs
- Scannables and detectors

<img src="img/gda.png" width="1000"/>

## Why are we Interested in Bluesky?

- Jython no longer supported
- Restricted to Python 2
- Hard to maintain a hybrid application, may be better to have a clean cut between Java and Python code
- Hardware triggered scanning

## Bluesky-as-a-Service

- Controlled via REST API/GraphQL API/Message Bus
- Takes commands and emits events
- Easy to add new plans

<img src="img/service.svg" width="500"/>

Would like to be able to write

```python
def my_special_plan(detectors: List[Readable], motor: Movable, number_of_iterations: int = 3):
    ...
```

And have it automatically converted to an endpoint

#### Prototyping Work
- NSLS-II: https://github.com/bluesky/bluesky-queueserver
- DLS: https://gitlab.diamond.ac.uk/daq/d2acq/services/bluesky-service

## Ophyd V2

- https://github.com/dls-controls/ophyd
- Early stages of development

The main aims of this new version are:

- Split into Devices (containing PV interface) and Abilities (containing the logic)

```python
@dataclass
class Motor:
    position: SignalRW[float]
    velocity: SignalRW[float]
    ...

class MovableMotor:
    motor: Motor
    
    async def set(self, pos: float) -> Status:
        awaitable = self.motor.position.put(pos)
        return Status(awaitable)
```

- Make all Abilities async
```python
await asyncio.wait([motor.set(3) for motor in [x, y, z]])
```
- Provide useful helpers for flyscans
    - Quickly build `Flyable`s with a few lines of code
    - Based on DLS Malcolm project
